In [ ]:
import wave
from pyaudio import PyAudio,paInt16
import os
from aip import AipSpeech
import ffmpy #视频音频转码模块
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import sys
from pydub import AudioSegment
import requests
from urllib import request
from bs4 import BeautifulSoup
import urllib
import time
from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as expected
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains


framerate=48000 #原来是8000，现在用了麦克风似乎它在pyaudio中只能使用48000的采样率，用shell命令可以支持8k采样率
NUM_SAMPLES=2000
channels=1
sampwidth=2
TIME=5
device = 'com'
APP_ID = '11169887'
API_KEY = 'TQypLIsDnr4XwzfyKGLqMsfD'
SECRET_KEY = 'bc5efee36b796c2b467dba12c2a080b0'


def save_wave_file(filename,data):
    '''save the date to the wavfile'''
    wf=wave.open(filename,'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(sampwidth)
    wf.setframerate(8000)
    wf.writeframes(b"".join(data))
    wf.close()

def my_record(device = 'com'): #树莓派版的录音函数，有获取录音设备号以及选择录音设备的语句
    pa=PyAudio()
    if device != 'com':
        info = p.get_host_api_info_by_index(0)
        numdevices = info.get('deviceCount')
        for i in range(0, numdevices):
            if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0 and re.match('USB', p.get_device_info_by_host_api_device_index(0, i).get('name')) is not None:
                input_device_index = i
                break
        stream=pa.open(format = paInt16,channels=1,
                       rate=framerate,input=True,
                       input_device_index = input_device_index,
                       frames_per_buffer=NUM_SAMPLES)
    else:
        stream=pa.open(format = paInt16,channels=1,
               rate=framerate,input=True,
               frames_per_buffer=NUM_SAMPLES)
    my_buf=[]
    count=0
    print('Recording:',end='')
    
    while count <= TIME * framerate // NUM_SAMPLES: #控制录音时间
        string_audio_data = stream.read(NUM_SAMPLES)
        if framerate != 8000 and framerate != 16000:
            ratio = round(framerate / 8000)            
            string_audio_data = bytes([rv for r in zip(string_audio_data[::sampwidth * ratio], string_audio_data[1::sampwidth * ratio]) for rv in r])
        my_buf.append(string_audio_data) #my_buf的每一个元素是一个长度为sampwidth * NUM_SAMPLES的bytes
        count+=1
        if (count % (framerate // NUM_SAMPLES) == 0):
            print('...',end='')
    print()
    stream.close()
    return my_buf

chunk=2048 #好像是分段记录时每次记录的点数
def play(filename): #可以播放mp3,wav,amr,mp4; pcm还不行，要额外指定sample_width 现在是单线程的，后面要研究一下如何变成多线程的，不然这个助理就用不了了
    try:
        sound = AudioSegment.from_file(filename, format=str.split(filename,'.')[1])
    except:
#         print('该wav文件缺少RIFF头')
        filename_answer = 'tmp.' + str.split(filename,'.')[1]
#         filename_answer = 'tmp.wav'
        if os.path.exists(filename_answer):
            os.remove(filename_answer)
        ff = ffmpy.FFmpeg( 
        inputs={filename: None},
        outputs={filename_answer: None}
        )
        ff.run() #美中不足的时没有学会怎么执行覆盖文件操作，文件存在时写入操作会被直接跳过，故现在只能暂时在转码播放后直接将其移除            
        sound = AudioSegment.from_file(filename_answer, format=str.split(filename_answer,'.')[1])        
    p=PyAudio()
    stream=p.open(format=p.get_format_from_width(sound.sample_width),channels= \
        sound.channels,rate=sound.frame_rate,output=True)
    i=0
    while True:
        data = sound.raw_data[i*chunk : (i+1)*chunk]
        if data==b'':break
        stream.write(data) #这一行是播放语音
        i = i + 1
    stream.close()
    p.terminate()       
    

# 读取文件
def get_file_content(filePath):
    with open(filePath, 'rb') as fp:
        return fp.read() 
    
def get_weather(message):
    date_heard = re.search(u'.*今天'.encode('utf8'), message)
    if date_heard is not None:
        day = 0
    else:
        date_heard = re.search(u'.*明天'.encode('utf8'), message)
        if date_heard is not None:
            day = 1
        else: 
            date_heard = re.search(u'.*后天'.encode('utf8'), message)
            if date_heard is not None:
                day = 2
    resp=urlopen('http://www.weather.com.cn/weather/101280101.shtml')
    soup=BeautifulSoup(resp,'html.parser')
    content = soup.findAll('li', class_="sky") #返回一个链表，每个元素是一个标签包括一天的天气内容
    tagToday=content[day].find('p', class_="tem")
    try:
        temperatureHigh=tagToday.span.string
    except AttributeError as e:
        temperatureHigh=tagToday.find_next('p', class_="tem").span.string
    temperatureLow=tagToday.i.string
    weather=soup.find('p', class_="wea").string
    tagWind=soup.find('p',class_="win")
    winL=tagWind.i.string
    time_now = datetime.datetime.now()
    delta = datetime.timedelta(days=day)
    time_day = time_now + delta
    date_string = str(time_day.month) + '月' + str(time_day.day) + '日' 
    name_date = ['今', '明', '后']
    result_weather = name_date[day] + '天是：'+ date_string + ',风级：' + winL + ',最低温度：' + temperatureLow + ',最高温度：' + temperatureHigh + ',天气：'+ weather
    return result_weather

def say_answer(message_answer): #client需要在外部创建一个公用的，因为每次向百度语音API申请一个太花费时间了
    result_tts = client.synthesis(message_answer, 'zh', 1, {'vol': 5, 'per': 3}) 
    if not isinstance(result_tts, dict):
        with open(filename_raw, 'wb') as f:
            f.write(result_tts)
    play(filename_raw)   

def listen_music(keyword):
# keyword = input("请输入想要听的歌曲：")
    url = "http://songsearch.kugou.com/song_search_v2?callback=jQuery1124006980366032059648_1518578518932&keyword="+keyword+"&page=1&pagesize=30&userid=-1&clientver=&platform=WebFilter&tag=em&filter=2&iscorrection=1&privilege_filter=0&_=1518578518934"
    content = requests.get(url)
    if re.findall('"FileHash":"(.*?)"',content.text) == []:
        message_answer = '没有搜索到对应的歌曲'
        result_tts = client.synthesis(message_answer, 'zh', 1, {'vol': 5, 'per': 0}) 
        with open(filename_answer, 'wb') as f:
            f.write(result_tts)
        play(filename_answer)
    else:
        filehash = re.findall('"FileHash":"(.*?)"',content.text)[0]
        songname = re.findall('"SongName":"(.*?)"',content.text)[0].replace("<\\/em>","").replace("<em>","") #即将播放的歌曲名
        hash_url = "http://www.kugou.com/yy/index.php?r=play/getdata&hash="+filehash
        hash_content = requests.get(hash_url)
        play_url = re.findall('"play_url":"(.*?)"',hash_content.text)
        play_url = ' '.join(play_url)
        real_download_url = play_url.replace("\\","")
        print("客官，请稍等一下，好音乐马上呈上！")
        # with open(songname+".mp3","wb")as fp:
        with open(filename_music,"wb")as fp:
            fp.write(requests.get(real_download_url).content)
        print('下载完成，敬请收听！')
        play(filename_music)
    

class Browser():
    def __init__(self):        
        self.options = Options()
        self.profile = webdriver.FirefoxProfile()
        self.profile.set_preference("general.useragent.override", "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:59.0) Gecko/20100101 Firefox/59.0")
        self.driver = webdriver.Firefox(firefox_profile=self.profile, options=self.options)
        self.window_handle_dianping = ''
    def Life_assistant(self,keyword):    
        if self.driver.current_url == 'about:blank':
            self.window_handle_blank = self.driver.current_window_handle
            if self.window_handle_dianping == '' or self.window_handle_dianping == self.window_handle_blank: #未定义不等于是空
                url = 'https://www.dianping.com/'
                self.driver.get(url)
                self.window_handle_dianping = self.driver.current_window_handle        

        self.driver.find_element_by_id('J-search-input').send_keys(keyword + Keys.CONTROL + 'a')
        self.driver.find_element_by_id('J-all-btn').click() #click之后弹出了新窗口，在我们看来新窗口在前面，但其实句柄还是原来的窗口  
        windows = self.driver.window_handles
        self.driver.switch_to.window(windows[-1])
        element = WebDriverWait(self.driver, 10).until(lambda x: x.find_element_by_tag_name('h4')) #要等到页面加载出下面的店铺才能进行读取，不然就读不到了
        links = self.driver.find_elements_by_tag_name('h4')
        list_links = []    
        for link in links[2:5]: #links[0和1]是其他无关信息    
            text = link.text
            if re.match(r'.*\(', link.text) is not None:
                text = link.text.split('(')[0] + '，' + link.text.split('(')[1][:-1]
            list_links.append(text)
        windows = self.driver.window_handles
        if len(windows) != 1: #保留一个主窗口，不然driver会被关掉
            self.driver.close() #跟打开新窗口一样，关闭当前句柄对应的窗口之后，句柄不会自己变化
            self.driver.switch_to.window(self.window_handle_dianping) #需要我们自行改回原窗口的句柄
        return list_links
    def close():
        self.driver.close()
    def quit():
        self.driver.quit()


    


In [ ]:
if device == 'com':
    os.chdir('E:\我的东西\新建文件夹\文件\课件\信工课件\电子系统综合设计\半成品\代码文件')
    filename_record = 'audio_record.wav'
    filename_answer = 'audio_answer.wav'
    filename_raw = 'audio_raw.wav'
    filename_music_name = 'AudioBase_DIY/music_name2.wav'
    filename_music = 'music.mp3'
    filename_ask_weather = 'AudioBase_DIY/ask_weather.wav'
    filename_ask_music = 'AudioBase_DIY/ask_music2.wav'
    filename_life_assist = 'AudioBase_DIY/nevigate_name3.wav'
elif device == 'pi':
    os.chdir('/home/pi/May/')
    filename_record = 'record/audio_record.wav'
    filename_answer = 'record/audio_answer.wav'
    filename_raw = 'record/audio_raw.wav'
    filename_music_name = 'sample/music_name2.wav'
    filename_music = 'record/music.mp3'
    filename_ask_music = 'sample/ask_music2.wav'


flag_debug = 1 #1表示处于调试状态
client = AipSpeech(APP_ID, API_KEY, SECRET_KEY) 
if 'browser' not in locals().keys(): #只打开一个浏览器，不重复打开
    browser = Browser()

# print('recording for 5 seconds...')
# os.system('sudo arecord -D "plughw:1,0" -f S16_LE -r 8000 -d 5 '+ filename_record) #这里暂时没有搞定python调用麦克风录音，就移花接木了一波，用shell命令来调用麦克风录音
data = my_record(device)
save_wave_file(filename_record,data)

# message = result_weather.encode('utf8')
# wf=wave.open(filename_record,'rb')
# rate=wf.getframerate()    #照目前的情况是8000，如果有变化，再用上这个语句
filename_debug = filename_life_assist
if flag_debug:
    result_stt = client.asr(get_file_content(filename_debug), 'wav', 8000, {'dev_pid': '1537'})
else:
    result_stt = client.asr(get_file_content(filename_record), 'wav', 8000, {'dev_pid': '1537'}) #dev_pid=1537是纯中文普通话，1536支持简单英文
#对wav文件无法识别没有RIFF头的文件，不能直接传data
# wf.close()

if result_stt['err_msg'] != 'success.':
    print('没有识别到内容')
else:
    print(result_stt['result'])
    message = str(result_stt['result'][0]).encode('utf8')
#     message = str(result_stt['result']).encode('utf8')
    search_result = re.search(u'.*魔镜'.encode('utf8'), message)
    if search_result is not None: #接收到了唤醒语，开始工作
        flag = 1
        search_result = re.search(u'.*天气'.encode('utf8'), message)
        if search_result is not None:
            message_answer = get_weather(message)
            say_answer(message_answer)
        else: 
            pattern = re.compile(u'歌|音乐'.encode('utf8'))
            search_result = re.findall(pattern, message)
            if len(search_result) != 0: #search失败返回的是Nonetype，findall失败返回一个空list
                message_answer = '你想听什么' + search_result[0].decode('utf8')
                say_answer(message_answer)
                data = my_record(device) #再接收一波语音
                save_wave_file(filename_record, data)
                if flag_debug:
                    result_stt = client.asr(get_file_content(filename_music_name), 'wav', 8000, {'dev_pid': '1537'})
                else:
                    result_stt = client.asr(get_file_content(filename_record), 'wav', 8000, {'dev_pid': '1537'})
#                 keyword = input('请输入想听的歌曲名：')
                keyword = result_stt['result'][0].replace('，','') #str类型即可
                print(keyword)
                say_answer('好的，请稍等')
#                 keyword = '告白气球'
                listen_music(keyword)                
            else: #生活助理
                pattern = re.compile(u'我想吃(.*)|我想看(.*)|我想去(.*)'.encode('utf8'))
                search_result = re.findall(pattern, message)
                for data in search_result[0]:
                    if len(data) != 0:
                        say_answer('正在为您搜索相关店铺')
                        stores = browser.Life_assistant(data.decode('utf8'))
                        print('为您搜索到以下店铺：')
                        say_answer('为您搜索到以下店铺：')
                        for store in stores:
                            print(store)
                            say_answer(store)
                        break
                else:
                    flag = 0 #现在这个flag好像没什么用了

    #         message_answer = '你在说什么我听不见'
#         if flag:
#             result_tts = client.synthesis(message_answer, 'zh', 1, {'vol': 5, 'per': 3}) 
#             if not isinstance(result_tts, dict):
#                 with open('audio_raw.wav', 'wb') as f:
#                     f.write(result_tts)
#             play('audio_raw.wav')

    # 首先实现：读取A23_0.wav，能将识别内容显示在屏幕上（已完成）
    # 然后实现：将显示在屏幕上的识别内容转成音频
    # 接着给音频加上RIFF头
    # 最后播放音频

In [ ]:
def get_proxy():
    '''到某网站上获取代理IP，代理IP是会改变的'''
    url = 'http://www.xicidaili.com/'
    header = {}
    #写入User Agent信息
    header['User-Agent'] = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0'
    #创建Request对象
    req = request.Request(url, headers=header)   
    #传入创建好的Request对象
    response = request.urlopen(req)
    soup=BeautifulSoup(response,'html.parser')
    links = soup.find_all('td')
    proxys = []
    for link in links:
#         print(re.match('[0-9]{4}', link.get_text()))
        if len(link.get_text().split('.')) == 4:
            ip_name = link.get_text()
        elif re.match('[0-9]{4}', link.get_text()) is not None:
            proxys.append((ip_name + ':' + link.get_text())) 

    return proxys

def fake_Life_assistant(keyword):
    '''使用代理服务器访问大众点评，但是似乎还是被拒了'''
    from urllib import request
    from bs4 import BeautifulSoup
    import urllib
    import random
    proxys = get_proxy()
    work_well = 1 #1为正常访问保存代理IP信息的网站，0为异常
    keyword = '日料'
    #访问网址
    url = 'https://www.dianping.com/search/keyword/4/0_' + urllib.parse.quote(keyword)
    #这是代理IP
    if work_well:
        proxy = {'http': proxys[random.randint(0, len(proxys)-1)]} #从爬下来的代理里面找一个
    else: #人工输入
        proxy = {'http': '119.28.194.66:8888'}
    #创建ProxyHandler
    proxy_support = request.ProxyHandler(proxy)
    #创建Opener
    opener = request.build_opener(proxy_support) 
    #添加User Angent
    opener.addheaders = [('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0')]
    #安装Opener
    # request.install_opener(opener) # 使用install_opener方法之后，会将程序默认的urlopen方法替换掉。也就是说，如果使用install_opener之后，在该文件中，再次调用urlopen会使用自己创建好的opener。如果不想替换掉，只是想临时使用一下，可以使用opener.open(url)，这样就不会对程序默认的urlopen有影响。
    # response = request.urlopen(url)
    response = opener.open(url)
    soup=BeautifulSoup(response,'html.parser')
    print(soup)

def fake_Life_assistant_2(keyword):
    '''根据关键字到大众店铺上搜索，并返回前三个店铺'''    
    from urllib import request
    from bs4 import BeautifulSoup
    import urllib
    url = 'https://www.dianping.com/search/keyword/4/0_' + urllib.parse.quote(keyword)
    header = {}
    header['User-Agent'] = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0'
    req = request.Request(url, headers=header)
    response = request.urlopen(req)
    soup=BeautifulSoup(response,'html.parser')
    links = soup.find_all('h4')
    list_links = []    
    for link in links[2:5]: #links[0和1]是其他无关信息    
        print(link)
        print(link.split('('))
        if link.split('(') is not None:
            link = link.split('(')[0] + '，(' + link.split('(')[1]
        list_links.append(link.get_text())
    return list_links     